In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'august'
new_month = 'september'

In [95]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='assets',index_col='Ticker')
#change call to new filepath
#coinmarketcap_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
#                         sheetname='coinmarketcap',index_col='Ticker')

In [96]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets?with-profiles"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [97]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.index = map(lambda x: x.lower(), df.index)

In [98]:
url_messari = "https://data.messari.io/api/v1/assets/btc/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)
df_all = pd.Series()

In [99]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Available Supply'] = (d_messari['data']['supply']['circulating'])
    df_all['Supply % Issued'] = df_all['Available Supply'] / (df_all['Y2050 Supply'])
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    #df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Available Supply']
    df_all['Category'] = (d_messari['data']['misc_data']['categories'][0])
    df_all['Sectors'] = (d_messari['data']['misc_data']['sectors'][0])
    try:
        df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 150)
    except:
        pass
    try:
        df_all['Category2'] = (d_messari['data']['misc_data']['categories'][1])
    except:
        pass
    try:
        df_all['Sectors2'] = (d_messari['data']['misc_data']['sectors'][1])
    except:
        pass
    return df_all

In [100]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Available Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        df.loc[t,('Category')] = metrics['Category']
        df.loc[t,('Sectors')] = metrics['Sectors']
        try:
            df.loc[t,('Est Volume')] = metrics['Est Volume']
        except:
            pass
        try:
            df.loc[t,('Category2')] = metrics['Category2']
        except:
            pass
        try:
            df.loc[t,('Sectors2')] = metrics['Sectors2']
        except:
            pass
    except:
        pass

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [101]:
#t = 'IOTA'
#metrics = get_messari_data(t)
#t = 'IOT'
#print (metrics)
#metrics_log[t] = metrics
#df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
#df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
#df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
#df.loc[t,('Price')] = metrics['Price']
#df.loc[t,('Marketcap')] = metrics['Marketcap']
#df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
#df.loc[t,('Category')] = metrics['Category']
#df.loc[t,('Sectors')] = metrics['Sectors']
#try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
#except:
#    pass
#try:
#    df.loc[t,('Category2')] = metrics['Category2']
#except:
#    pass
#try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
#except:
#    pass

In [102]:
t = 'VET'
metrics = get_messari_data(t)
metrics_log[t] = metrics
df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
df.loc[t,('Price')] = metrics['Price']
df.loc[t,('Marketcap')] = metrics['Marketcap']
df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
df.loc[t,('Category')] = metrics['Category']
df.loc[t,('Sectors')] = metrics['Sectors']
try:
    df.loc[t,('Est Volume')] = metrics['Est Volume']
except:
    pass
try:
    df.loc[t,('Category2')] = metrics['Category2']
except:
    pass
try:
    df.loc[t,('Sectors2')] = metrics['Sectors2']
except:
   pass

In [103]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [104]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [105]:
#read in other data sources
onchainfx_db = df

In [106]:
#remove duplicates
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
#coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

146
146
145
145


In [107]:
#fix any ticker mapping issues - rank date use these
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'MIOTA'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'MIOTA'})
asset_db = asset_db.rename({'IOTA' : 'MIOTA'})

In [108]:
# #fix any ticker mapping issues - rebalance day use these
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'IOTA' : 'IOT'})
# asset_db = asset_db.rename({'IOTA' : 'IOT'})

In [109]:
onchainfx_db

Coin  Class  Sector  Quality Exchange?  ERC20  ICO ok?  \
ADA            Cardano    2.0     NaN                1.0    0.0      1.0   
AE           Aeternity    3.0     NaN                1.0    1.0      1.0   
AION              Aion    2.0     NaN                1.0    1.0      1.0   
ANT             Aragon    3.0     NaN                1.0    1.0      1.0   
ARDR             Ardor    2.0     NaN                1.0    0.0      1.0   
ARK                Ark    2.0     NaN                1.0    0.0      1.0   
BAT                BAT    3.0     NaN                1.0    1.0      1.0   
BCD    Bitcoin Diamond    1.0     NaN                1.0    0.0      1.0   
BCH       Bitcoin Cash    1.0     NaN                1.0    0.0      1.0   
BCN           Bytecoin    1.0     NaN                1.0    0.0      0.0   
BLOCK         blocknet    2.0     NaN                1.0    0.0      1.0   
BLZ           Bluzelle    2.0     NaN                1.0    0.0      1.0   
BNB       Binance Coin    3.0     NaN                1.0    1.0      1.0   
BNT             Bancor    3.0     NaN                1.0    1.0      1.0   
BSV         Bitcoin SV    1.0     NaN                1.0    0.0      1.0   
BTC            Bitcoin    1.0     NaN                1.0    0.0      1.0   
BTCP   Bitcoin Private    1.0     NaN                0.0    0.0      1.0   
BTG       Bitcoin Gold    1.0     NaN                1.0    0.0      1.0   
BTM              Bytom    2.0     NaN                0.0    0.0      1.0   
BTS          BitShares    2.0     NaN                1.0    0.0      1.0   
CENNZ       Centrality    3.0     NaN                0.0    0.0      1.0   
CND         Cindicator    3.0     NaN                1.0    0.0      1.0   
CNX          Cryptonex    3.0     NaN                0.0    0.0      1.0   
CTXC            Cortex    2.0     NaN                0.0    0.0      1.0   
CVC              Civic    3.0     NaN                1.0    1.0      1.0   
DASH              Dash    1.0     NaN                1.0    0.0      1.0   
DCN          Dentacoin    3.0     NaN                1.0    1.0      1.0   
DCR             Decred    2.0     NaN                1.0    0.0      1.0   
DENT              Dent    3.0     NaN                0.0    0.0      1.0   
DGB           DigiByte    1.0     NaN                1.0    0.0      1.0   
...                ...    ...     ...                ...    ...      ...   
VET            VeChain    2.0     NaN                1.0    1.0      1.0   
VTC           Vertcoin    1.0     NaN                1.0    0.0      1.0   
WAN           Wanchain    2.0     NaN                1.0    0.0      1.0   
WAVES            Waves    2.0     NaN                1.0    0.0      1.0   
WINGS            Wings    2.0     NaN                1.0    1.0      1.0   
WTC        Waltonchain    2.0     NaN                1.0    1.0      1.0   
XEM                NEM    2.0     NaN                1.0    0.0      1.0   
XLM            Stellar    1.0     NaN                1.0    0.0      1.0   
XMR             Monero    1.0     NaN                1.0    0.0      1.0   
XRP             Ripple    2.0     NaN                1.0    0.0      1.0   
XTZ              Tezos    2.0     NaN                1.0    0.0      1.0   
XVG              Verge    1.0     NaN                1.0    0.0      1.0   
XZC              ZCoin    1.0     NaN                1.0    0.0      1.0   
ZCL           Zclassic    1.0     NaN                1.0    0.0      1.0   
ZEC              Zcash    1.0     NaN                1.0    0.0      1.0   
ZEN            Zencash    1.0     NaN                1.0    0.0      1.0   
ZIL            Zilliqa    2.0     NaN                1.0    1.0      1.0   
ZRX                 0x    2.0     NaN                1.0    1.0      1.0   
RVN          RavenCoin    2.0     NaN                1.0    0.0      1.0   
HT         Huobi Token    3.0     NaN                0.0    1.0      1.0   
HOT          Holochain    2.0     NaN                1.0    0.0   

In [110]:
#onchainfx_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\ocfx_'+new_month+'_2019.xlsx',\
#index_col='Ticker')

In [111]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [112]:
#main function to create table of all assets
#def create_universe(onchainfx,coinmarketcap,assets):
#    eligibility = pd.DataFrame(columns = ['Marketcap'])
#    eligibility['Marketcap'] = onchainfx['Marketcap']
#    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
#    eligibility['Float Percent'] = onchainfx['Supply % Issued']
#    eligibility['Float'] = onchainfx['Available Supply']
#    eligibility['Price'] = onchainfx['Price']
#    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
#    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
#    for t in eligibility.index:
#        if t in coinmarketcap.index:
#            eligibility.loc[t,('Volume')] = coinmarketcap.loc[t,('Volume (30d)')]
#        else:
#            print (t,' no volume data!')
#            eligibility.loc[t,('Volume')] = 1
#    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
#    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
#    for t in eligibility.index:
#        if t in assets.index:
#            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
#            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
#            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
#        else:
#            print (t,' no other data!')
#            eligibility.loc[t,('Other Pass')] = 0
#    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
#    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
#    check if duplicates
#    count = len(eligibility.index)
#    count1 = len(eligibility.index.unique())
#    if count != count1:
#        print ('duplicate tickers')
#    return eligibility

In [113]:
#main function to create table of all assets
def create_universe(onchainfx,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Volume'] = onchainfx['Est Volume']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [114]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [115]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

In [117]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check
ineligible_db

Marketcap  Current Marketcap  Float Percent         Float  \
ANT    2.597732e+07       1.964672e+07       0.756303  2.995679e+07   
BCN    1.141133e+08       1.138655e+08       0.997828  1.840668e+11   
BLOCK  2.961058e+07       8.223294e+06       0.277715  6.052445e+06   
BLZ             NaN                NaN            NaN           NaN   
BTCP   2.562703e+07       5.772568e+06       0.225253  4.733586e+06   
BTM             NaN                NaN            NaN           NaN   
CENNZ  1.850941e+07       1.246961e+07       0.673690  8.084282e+08   
CND             NaN                NaN            NaN           NaN   
CNX             NaN                NaN            NaN           NaN   
CTXC   2.793450e+07       1.647471e+07       0.589762  1.497925e+08   
DCN    2.470872e+08       1.004371e+07       0.040648  3.251875e+11   
DENT   5.166883e+07       3.797960e+07       0.735058  7.350584e+10   
DGD             NaN                NaN            NaN           NaN   
DRGN   2.258987e+07       1.242427e+07       0.549993  2.384219e+08   
ECN             NaN                NaN            NaN           NaN   
EDGE            NaN                NaN            NaN           NaN   
EMC2   1.842282e+07       1.344935e+07       0.730038  2.188598e+08   
ETN    0.000000e+00       3.628614e+07            inf  9.787784e+09   
GBYTE           NaN                NaN            NaN           NaN   
GRIN   2.229752e+09       4.377949e+07       0.019634  1.918464e+07   
HC              NaN                NaN            NaN           NaN   
HSR             NaN                NaN            NaN           NaN   
KCS    1.509032e+08       1.330447e+08       0.881656  8.816556e+07   
KIN             NaN                NaN            NaN           NaN   
MCAP            NaN                NaN            NaN           NaN   
MET             NaN                NaN            NaN           NaN   
MKR    4.839319e+08       4.855600e+08       1.003364  1.000000e+06   
NMR    4.355482e+07       6.354734e+06       0.145902  1.349328e+06   
PLR             NaN                NaN            NaN           NaN   
POT    7.866671e+06       1.645797e+06       0.209211  2.226561e+08   
R      3.683932e+07       3.683932e+07       1.000000  4.844500e+08   
RDD    1.866289e+08       3.843922e+07       0.205966  2.880871e+10   
RHOC   4.534155e+07       1.968010e+07       0.434041  3.737899e+08   
SMART           NaN                NaN            NaN           NaN   
SNGLS           NaN                NaN            NaN           NaN   
SPANK  7.368373e+06       2.210832e+06       0.300043  3.000434e+08   
SUB    9.389747e+06       6.075118e+06       0.646995  3.830210e+08   
WINGS           NaN                NaN            NaN           NaN   
ZCL             NaN                NaN            NaN           NaN   
HT     2.328358e+09       2.328367e+08       0.100000  5.000020e+07   
HOT    2.078512e+08       1.107552e+08       0.532858  1.332146e+11   
NEXO   8.631060e+07       4.833394e+07       0.560000  5.600000e+08   
ELA    9.152264e+07       4.424480e+07       0.483430  1.595319e+07   
PAI             NaN                NaN            NaN           NaN   
ATOM            NaN                NaN            NaN           NaN   

            Price  Total Supply        Volume  Float Pass  Volume Ratio  \
ANT      0.655835  3.960952e+07  7.235638e+05           1      0.036829   
BCN      0.000619  1.844675e+11  1.420065e+06           1      0.012471   
BLOCK    1.358673  2.179375e+07  3.966073e+05           1      0.048230   
BLZ           NaN           NaN           NaN           0           NaN   
BTCP     1.219492  2.101452e+07  3.860512e+06           1      0.668769   
BTM           NaN           NaN           NaN           0           NaN   
CENNZ    0.015425  1.200000e+09  6.028944e+05           1      0.048349   
CND           NaN           NaN           NaN           0           NaN   
CNX           NaN           NaN           NaN           0           

In [118]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [119]:
universe = universe.rename({'HSR' : 'HC'})
universe = universe.rename({'MIOTA' : 'IOT'})
universe = universe.rename({'VEN' : 'VET'})
universe

Marketcap  Current Marketcap  Float Percent         Float  \
BTC    2.170835e+11       1.851383e+11       0.852844  1.789806e+07   
XRP    2.695303e+10       1.156542e+10       0.429095  4.290954e+10   
ETH    2.537898e+10       2.018533e+10       0.795356  1.074810e+08   
XLM    9.869813e+09       1.362106e+09       0.138007  1.963614e+10   
BCH    6.498483e+09       5.565024e+09       0.856357  1.796846e+07   
LTC    6.145915e+09       4.624594e+09       0.752466  6.311729e+07   
EOS    4.889180e+09       3.324343e+09       0.679939  9.290762e+08   
BSV    2.730196e+09       2.323291e+09       0.850961  1.785499e+07   
BNB    2.560885e+09       3.983117e+09       1.555367  1.555367e+08   
XTZ    2.218577e+09       7.615600e+08       0.343265  6.603736e+08   
ADA    2.074100e+09       1.279938e+09       0.617105  2.592707e+10   
LINK   2.062055e+09       7.217192e+08       0.350000  3.500000e+08   
TRX    1.837986e+09       1.176826e+09       0.640280  6.668207e+10   
XMR    1.785863e+09       1.365292e+09       0.764500  1.717531e+07   
DASH   1.619309e+09       8.280394e+08       0.511353  9.000233e+06   
ETC    1.413073e+09       8.206153e+08       0.580731  1.129936e+08   
ZEC    1.071793e+09       3.726824e+08       0.347719  7.275744e+06   
NEO    9.802491e+08       6.914562e+08       0.705388  7.053883e+07   
ICX    9.586816e+08       1.145114e+08       0.119447  4.903925e+08   
ONT    7.980028e+08       4.257211e+08       0.533483  5.334832e+08   
RVN    7.485307e+08       1.543713e+08       0.206232  4.280345e+09   
IOT    7.476886e+08       7.476886e+08       1.000000  2.779530e+09   
DOGE   7.205479e+08       3.104858e+08       0.430902  1.209399e+11   
MAID   6.949241e+08       7.313711e+07       0.105245  4.525524e+08   
DCR    5.168612e+08       2.579881e+08       0.499144  1.029222e+07   
XEM    4.911594e+08       4.911594e+08       1.000000  9.000000e+09   
VET    3.946647e+08       2.523867e+08       0.639496  5.545473e+10   
MANA   3.196902e+08       3.733968e+07       0.116800  1.050142e+09   
BAT    2.805198e+08       2.392321e+08       0.852817  1.279226e+09   
LSK    2.798962e+08       1.446119e+08       0.516662  1.196273e+08   
...             ...                ...            ...           ...   
ENG    5.827223e+07       2.907247e+07       0.498908  7.483617e+07   
AION   5.763680e+07       3.022253e+07       0.524362  3.361658e+08   
LRC    5.157604e+07       3.369633e+07       0.653333  8.983047e+08   
IOTX   4.960760e+07       2.043833e+07       0.412000  4.120000e+09   
STRAT  4.646908e+07       4.060937e+07       0.873901  9.946088e+07   
CVC    4.558819e+07       1.562307e+07       0.342700  3.427000e+08   
POLY   4.052584e+07       1.775969e+07       0.438231  4.382314e+08   
EDO    3.692932e+07       2.192132e+07       0.593602  5.300173e+07   
TNT    3.535360e+07       1.514835e+07       0.428481  4.284813e+08   
FUN    3.520419e+07       2.095891e+07       0.595353  6.548879e+09   
FCT    3.461530e+07       3.842806e+07       1.110147  9.708346e+06   
KNC    3.321915e+07       3.127378e+07       0.941438  1.677761e+08   
LOOM   3.285783e+07       3.185419e+07       0.969455  9.694550e+08   
BNT    3.061083e+07       2.424554e+07       0.792058  6.284502e+07   
PPT    2.988629e+07       2.988629e+07       1.000000  5.325225e+07   
MTL    2.787914e+07       2.008965e+07       0.720598  4.798382e+07   
PAY    2.459463e+07       1.400117e+07       0.569277  1.168261e+08   
VTC    2.404254e+07       1.461313e+07       0.607803  5.095677e+07   
SYS    2.315474e+07       1.528531e+07       0.660137  5.615472e+08   
ETHOS  2.023970e+07       8.888349e+06       0.439154  9.762190e+07   
RDN    1.888338e+07       9.553845e+06       0.505939  5.059394e+07   
RCN    1.757530e+07       8.913129e+06       0.507139  5.071394e+08   
NXT    1.724200e+07       1.724200e+07       1.000000  9.989999e+08   
STORM  1.672102e+07       1.042338e+07       0.623370  6.233700e+09   
LBC    1.611085e+07       3.909771

In [120]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained ADA
New add BNB
10 full! Do not add  XTZ
10 full! Do not add  LINK


In [121]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained VET
Retained MANA
Retained BAT
Retained LSK
20 full! No room for BTG
20 full! Do not add  QTUM
20 full! Do not add  SC
20 full! Do not add  DGB


In [122]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained XVG
Retained ELF
Retained MCO
Retained WAX
Retained NAS
Retained ARDR
Retained STORJ
Retained PIVX
Retained GNT
Retained POWR
40 full! No room for ENG
40 full! No room for AION
40 full! No room for LRC
40 full! No room for STRAT
40 full! Do not add  IOTX
40 full! Do not add  CVC
40 full! Do not add  POLY
40 full! Do not add  EDO
40 full! Do not add  TNT
40 full! Do not add  FUN


In [123]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [124]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [125]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

Currency adds ['DGB' 'IOT' 'ZEN']
Currency deletes ['MIOTA']
Currency count members: 19

20 adds ['DCR' 'IOT' 'TRX']
20 deletes ['BNB' 'BTG' 'MIOTA']
20 count members: 20

ETH adds []
ETH deletes ['AION' 'ENG' 'LRC']
ETH count members: 20

Platform adds ['DCR' 'WAN']
Platform deletes ['AION' 'STRAT']
Platform count members: 28

10 adds ['BNB']
10 deletes ['TRX']
10 count members: 10

40 adds ['BTG' 'DGB' 'GXC' 'WAN' 'ZEN']
40 deletes ['AION' 'ENG' 'GXS' 'LRC' 'STRAT']
40 count members: 40

Application adds ['GXC']
Application deletes ['ENG' 'GXS' 'LRC']
Application count members: 23

Total adds ['DCR' 'DGB' 'GXC' 'IOT' 'WAN' 'ZEN']
Total deletes ['AION' 'ENG' 'GXS' 'LRC' 'MIOTA' 'STRAT']
Total count members: 70

Sector count members: 70


In [126]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])